In [1]:
!pip install torchviz
!pip install graphviz

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 315.8 kB/s eta 0:00:00MB/s eta 0:00:01
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4132 sha256=38de05a173f2ad425a3ecee1265524b88f93cf634776a50e2634af89c2f5b35f
  Stored in directory: /home/juanessao2000/.cache/pip/wheels/5a/d0/3f/b7014553eb74f12892b7d9b69c6083044564712d10fde8dfdc
Successfully built torchviz


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

class InvModel1(nn.Module):
    def __init__(self, in_shape, out_shape, hidden_units):
        super().__init__()
        padding = 1
        self.simple_conv = nn.Sequential(
            nn.Conv1d(in_channels=in_shape, out_channels=600, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Conv1d(in_channels=600, out_channels=1200, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Conv1d(in_channels=1200, out_channels=2400, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Conv1d(in_channels=2400, out_channels=4800, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Flatten(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=326400, out_features=out_shape)
        )
        
    def forward(self, x):
        return self.simple_conv(x)

# Instantiate the model
model_0 = InvModel1(300, 4*20, 4096).float()

# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("The model will be run on:", device)

# Move the model to the appropriate device
model_0.to(device)

# Create a dummy input tensor with the correct shape (batch_size, in_channels, sequence_length)
dummy_input = torch.randn(80, 300, 64).to(device)

# Define the optimizer and loss function
optimizer = optim.Adam(model_0.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

# Forward pass to create a graph
output = model_0(dummy_input)

# Create a visualization of the model
dot = make_dot(output, params=dict(model_0.named_parameters()), show_attrs=True, show_saved=True)

# Save the visualization to a file
dot.format = 'png'
file_path = 'InvModel1_graph_torchviz'
dot.render(file_path)

The model will be run on: cuda


'InvModel1_graph_torchviz.png'

In [6]:
import torch
import torch.nn as nn
from graphviz import Digraph

class InvModel1(nn.Module):
    def __init__(self, in_shape, out_shape, hidden_units):
        super().__init__()
        padding = 1
        self.simple_conv = nn.Sequential(
            nn.Conv1d(in_channels=in_shape, out_channels=600, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Conv1d(in_channels=600, out_channels=1200, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Conv1d(in_channels=1200, out_channels=2400, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Conv1d(in_channels=2400, out_channels=4800, kernel_size=2, stride=1, padding=padding),
            nn.ReLU(),
            nn.Flatten(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=38400, out_features=out_shape)
        )
        
    def forward(self, x):
        return self.simple_conv(x)

# Instantiate the model
model_0 = InvModel1(300, 4*20, 4096).float()

# Visualize the model using graphviz
def visualize_model(model):
    dot = Digraph()
    dot.node(name='input', label='Input Layer')

    for idx, layer in enumerate(model.simple_conv):
        if isinstance(layer, nn.Conv1d):
            dot.node(name=f'conv{idx}', label=f'Conv1D\n{layer.in_channels} -> {layer.out_channels}')
            dot.edge(f'conv{idx-1}' if idx > 0 else 'input', f'conv{idx}')
        elif isinstance(layer, nn.ReLU):
            dot.node(name=f'relu{idx}', label='ReLU')
            dot.edge(f'conv{idx-1}', f'relu{idx}')
        elif isinstance(layer, nn.Flatten):
            dot.node(name='flatten', label='Flatten')
            dot.edge(f'relu{idx-1}', 'flatten')
        elif isinstance(layer, nn.Dropout):
            dot.node(name='dropout', label='Dropout')
            dot.edge('flatten', 'dropout')
        elif isinstance(layer, nn.Linear):
            dot.node(name='linear', label=f'Linear\n{layer.in_features} -> {layer.out_features}')
            dot.edge('dropout', 'linear')

    dot.node(name='output', label='Output Layer')
    dot.edge('linear', 'output')

    return dot

# Generate and save the visualization
dot = visualize_model(model_0)
dot.format = 'png'
file_path = 'InvModel1_graph'
dot.render(file_path)

file_path + '.png'

'InvModel1_graph.png'